In [1]:
import copy
import pandas as pd
import numpy as np
import Levenshtein
from tqdm import tqdm

In [2]:
! pwd
mf=pd.read_csv('../../../../paper_data_v3/irat_enet/results/animal_predictions/animal_combined_with_irat.csv')#.assign(is_irat=0)
mf=mf.rename(columns={'name':'id','ha_sequence':'ha','na_sequence':'na','emergence':'emergence_risk'}).assign(impact_risk=0)
mf.to_csv('v3_animalrisk.csv')

/home/ishanu/Dropbox/ZED/Research/emergenet/paper_data_v1/irat_qnet/results/animal_predictions


In [3]:
# parameters
threshold=6.25  # draw tree above this emergenec risk threshold
CONSTRUCT_PHYLO=False  # use constructed tree (tree construction takes hours)
OUTPUT_DIR='./'
PHYLO_TREE_DIR='./'
PHYLO_DIR='./'
COMBINED_RESULTS='v3_animalrisk.csv'
#COMBINED_RESULTS='/home/ishanu/Downloads/combined_results_irat.csv'
#CONSTRUCT_PHYLO=False
num_collapsed=19  # number of mutations within which leaves are collapsed
VERBOSE=False

In [4]:
mf

,id,subtype,date,ha_accession,ha,na_accession,na,ha_risk,na_risk,geom_mean_risk,...,emergence_low,emergence_high,is_irat,emergence_score,impact_score,mean_low_acceptable_emergence,mean_high_acceptable_emergence,mean_low_acceptable_impact,mean_high_acceptable_impact,impact_risk
0,A/Hong Kong/125/2017,H7N9,2017-05-01,EPI977395,MNTQILVFALIAIIPTNADKICLGHHAVSNGTKVNTLTERGVEVVN...,EPI977394,MNPNQKILCTSATAITIGAIAVLIGIANLGLNIGLHLKPGCNCSHS...,0.000010,0.000010,0.000010,...,6.187663,9.179121,1,6.5,7.5,5.65,7.51,6.74,8.50,0
1,A/Shanghai/02/2013,H7N9,2016-04-01,EPI448936,MNTQILVFALIAIIPTNADKICLGHHAVSNGTKVNTLTERGVEVVN...,EPI448938,MNPNQKILCTSATAIIIGAIAVLIGMANLGLNIGLHLKPGCNCSHS...,0.000010,0.000010,0.000010,...,6.187663,9.179121,1,6.4,7.2,5.52,7.43,6.41,8.32,0
2,A/California/62/2018,H1N2,2019-07-01,EPI1311361,MKVKLMVLLCTFTATYADTICVGYHANNSTDTVDTVLEKNVTVTHS...,EPI1311360,MNPNQKIITIGSISLTLAAMCFLMQTAILVTNVTLHFNQCECHYPP...,0.000010,0.000010,0.000010,...,6.187663,9.179121,1,5.8,5.7,4.22,7.16,3.80,7.09,0
3,A/Indiana/08/2011,H3N2,2012-12-01,EPI344405,MKTIIAFSCILCLIFAQKLPGSDNSMATLCLGHHAVPNGTLVKTIT...,EPI344404,MNPNQKIITIGSVSLIIATICFLMQIAILVTTVTLHFKQHDYNSPP...,0.000055,0.000080,0.000066,...,6.104862,9.073535,1,6.0,4.5,-1.00,-1.00,-1.00,-1.00,0
4,A/Sichuan/06681/2021,H5N6,2021-10-01,EPI1883261,MENIVLLLAIVSLVKSDQICIGYHANNSTEQVDTIMEKNVTVTHAQ...,EPI1883263,MNPNQKITCISATGVTLSIVSLLIGITNLGLNIGLHYKVSDSTTIN...,0.000010,0.194167,0.001393,...,5.153469,7.860356,1,5.3,6.3,3.88,6.45,5.04,7.47,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6372,A/chicken/China/2106/2021,H9N2,2021-11-30,EPI2116853,METASLITVLLVVTVSNADKICIGYQSTNSTETVDTLTENNVPVTH...,EPI2116851,MNPNQKIIAIGSVSLTIAIICFLMQIAILTTTMTLHFRQNECSNPS...,0.151614,0.045829,0.083357,...,2.164810,4.049335,0,NaN,NaN,NaN,NaN,NaN,NaN,0
6373,A/chicken/China/2034/2020,H9N2,2020-01-20,EPI2116765,METVSLITILLVAAVSNADKICIGYQSTNSTETVDTLTENNVPVTH...,EPI2116763,MNPNQKIIAIGSVSLTIAIICFLMQIAILTTTMTLHFRQNECSNPS...,0.152940,0.045829,0.083720,...,2.161398,4.044984,0,NaN,NaN,NaN,NaN,NaN,NaN,0
6374,A/chicken/China/2040/2020,H9N2,2020-01-10,EPI2116781,METVSLITILLVAAVSNADKICIGYQSTNSTETVDTLTENNVPVTH...,EPI2116779,MNPNQKIIAIGSVSLTIAIICFLMQIAILTTTMTLHFRQNECSNPS...,0.152942,0.045829,0.083721,...,2.161392,4.044977,0,NaN,NaN,NaN,NaN,NaN,NaN,0
6375,A/green-winged-teal/Georgia/DT-22246/2020,H5N8,2020-09-29,EPI1924318,MENIVLLLAIVSLVKSDQICIGYHANNSTEQVDTIMEKNVTVTHAQ...,EPI1924320,MNPNQKIATIGSISLGLVVFNVLLHALSIILMVLALGKSENNGICK...,0.248596,0.028458,0.084110,...,2.157757,4.040341,0,NaN,NaN,NaN,NaN,NaN,NaN,0


In [5]:
tmpf=pd.read_csv(COMBINED_RESULTS)
IRATSEQ=tmpf[tmpf.is_irat==1].id.values.astype(str)

# class definition to hold multi sequence data

In [6]:
class SeqInfo(object):
    """Holds information regarding the sequence.
    
    """
    def __init__(self, seq, 
                 protein,
                 accession,
                 subtype=None,
                 id=None,
                 name=None,
                 host=None, 
                 date=None, 
                 erisk=None,
                 irisk=None,
                 risk_flag=None,
                 country=None):
        self.name = name
        self.id = id
        self.protein=protein
        self.subtype=subtype        
        self.seq = seq
        self.accession = accession 
        self.host = host
        self.date = date
        self.erisk = erisk
        self.irisk = irisk
        self.risk_flag = risk_flag
        self.country = country
        
class MultipleSeqInfo(object):
    """Holds information regarding multiple sequences.
    
    Args:
        dataframe (pandas.DataFrame): list of records parsed from NCBI
        accessionname (str): column name for accession id
        proteinname (str): protein name 
        risk_threshold (float): emergence risk threshold to compute distance matrix
    """
    def __init__(self,
                 dataframe,
                 accessionname,
                 proteinname,
                 risk_threshold=6.2):
        
        self.seq_infos = {}
        self.risk_threshold = risk_threshold
        for i in np.arange(dataframe.index.size):
            record=dataframe.iloc[i,:]
            seqinfo = SeqInfo(
                name=record.id,
                seq=record[proteinname], 
                protein=proteinname,
                accession=record[accessionname],
                subtype=record.subtype,
                erisk=record.emergence_risk,
                irisk=record.impact_risk,
                risk_flag = record.emergence_risk > self.risk_threshold,
                host=None,
                date=None,
                country=None)
            #print(record.predicted_emergence_score > self.risk_threshold)
            self.seq_infos[seqinfo.accession] = seqinfo
            
    
    def compute_L_distance_matrix(self):
        highriskseq = pd.DataFrame.from_dict({key:val.seq 
                                              for (key,val) in self.seq_infos.items() 
                                              if val.risk_flag},orient='index',columns=['seq'])
        num=highriskseq.index.size
        d=np.zeros([num,num])
        for x in tqdm(np.arange(num*num)):
            j=x//num
            i=x-num*j
            if i > j:
                d[i,j] = Levenshtein.distance(highriskseq.seq.values[i],
                                                  highriskseq.seq.values[j])
        ds=pd.DataFrame(d)        
        ds=(ds+ds.transpose())
        ds.columns=highriskseq.index.values
        self.highriskdistancematrix=ds.copy()
        
        self.highriskdistancematrix.to_csv('dm'+str(self.risk_threshold)+'.csv',index=None)
        return 
    
    
    def accessions_to_subtype(self, accessions):
        """Create a dictionary mapping the accession to the host.
        """
        
        subtypes = []
        for accession in accessions:
            seqinfo = self.seq_infos[accession]
            subtypes.append(seqinfo.subtype)
            
        return subtypes

    def accessions_to_host(self, accessions):
        """Create a dictionary mapping the accession to the host.
        """
        
        hosts = []
        for accession in accessions:
            seqinfo = self.seq_infos[accession]
            hosts.append(seqinfo.host)
        return hosts
           
    

In [7]:
df=pd.read_csv(COMBINED_RESULTS,index_col=0).reset_index()
ALLinfoHA=MultipleSeqInfo(df.reset_index(),'ha_accession','ha',risk_threshold=threshold)

In [8]:
df[df.emergence_risk>threshold].subtype.value_counts()

H5N1    1732
H3N2     930
H1N1     481
H1N2     418
H3N8      14
H3N1       7
H7N9       2
H3N3       2
H5N6       1
H9N2       1
H5N2       1
H3N6       1
Name: subtype, dtype: int64

# generate distance matrix

In [9]:
ALLinfoHA.compute_L_distance_matrix()

  2%|▌                             | 237104/12880921 [00:08<07:22, 28582.03it/s]


KeyboardInterrupt: 

In [10]:
import dill as pickle
# Pickling (serializing) the object
with open('ALLinfo.pkl', 'wb') as file:
    pickle.dump(ALLinfoHA, file)

# Unpickling (deserializing) the object
with open('ALLinfo.pkl', 'rb') as file:
    ALLinfoHA = pickle.load(file)

# tree construction from distance matrix using biopython

In [11]:
from Bio.Phylo import TreeConstruction
from Bio import Phylo
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from Bio.Align import MultipleSeqAlignment
from Bio import Entrez
from Bio import SeqIO


def load_dm(file_, upper_diag=True):
    """Load the distance matrix. 
    
    Also, do some preprocessing. 
    """
    
    df = pd.read_csv(file_)
    #df.set_index('Unnamed: 0', inplace=True)
    #assert np.all(df.columns == df.index)
    
    # drop duplicate columns after reading csv
    #df = df.loc[:, ~df.columns.str.replace("(\.\d+)$", "").duplicated()]
    
    if upper_diag:
        df = df + df.T
    return df

def save_tree(tree, file_name, save_type='xml'):
    """Saved the created phylogenetic tree."""
    
    if save_type == 'pickle':
        graph = Phylo.to_networkx(tree)
        save_pickled(graph, file_name)
    elif save_type == 'xml':
        Phylo.write(tree, file_name, 'phyloxml')
    else:
        raise ValueError('Not a correct save type.')
    
def pandas_dm_to_biopython_dm(dm):
    """Convert the pandas distance matrix to the biopython distance matrix.
    
    Returns:
        biopython distance matrix
    """
    
    accessions = dm.columns
    bio_dm = []
    for i, accession in enumerate(accessions):
        bio_dm.append(list(dm.iloc[i, :i+1].values))
        
    bio_dm = TreeConstruction._DistanceMatrix(
        list(dm.columns), 
        bio_dm)
    
    return bio_dm

def distance_matrix_to_phylo_tree(dm, outfile=None):
    """Create a phylogenetic tree from the distance matrix."""
    
    dm = pandas_dm_to_biopython_dm(dm)
    
    treeConstructor = TreeConstruction.DistanceTreeConstructor()
    tree = treeConstructor.nj(dm)
    
    if outfile is not None:
        save_tree(tree, outfile)

# ete3 function, not all of these are used here

In [12]:
from ete3 import Tree, TreeStyle
from ete3 import Phyloxml
from ete3 import AttrFace, faces, Tree, NodeStyle, TreeStyle

def load_pickled(file_name):
    with open(file_name, 'rb') as f:
        return pickle.load(f, encoding='latin')


def get_farthest_node(tree, sequence):
    return (tree&sequence).get_farthest_node()

def get_all_accessions_from_tree(tree):
    return [leaf_node.name for leaf_node in tree.get_leaves()]

def remove_certain_hosts_from_tree(tree, hosts):
    """Remove leaf nodes if the host of that leaf is in `hosts`"""
    
    tree = copy.deepcopy(tree)
    
    removed_accessions = []
    for leaf_node in tree.get_leaves():
        if leaf_node.host in hosts:
            leaf_node.detach()
            
    return tree

def set_midpoint_outgroup(tree):
    tree.set_outgroup(tree.get_midpoint_outgroup())


def load_tree(filename, type_='phyloxml'):
    """Load saved phylogenetic tree.
    """
    
    if type_ == 'phyloxml':
        project = Phyloxml()
        project.build_from_file(filename)

        for tree in project.get_phylogeny():
            break

        t=tree
        
    elif type_ == 'newick':
        t = Tree(filename, format=1)
    else:
        raise ValueError('Not a correct type.')
    
    return t

# construct tree from distance matrix

In [13]:
#CONSTRUCT_PHYLO=True
if CONSTRUCT_PHYLO:
    ALL_dm_ldistance = load_dm(
        OUTPUT_DIR + 'dm'+str(threshold)+'.csv', 
        upper_diag=False)
    
    distance_matrix_to_phylo_tree(
        ALL_dm_ldistance, PHYLO_TREE_DIR + 'ldistance'+str(threshold)+'.xml')

# convert phyloxml tree to newick tree to manipulate trees

In [14]:

Phylo.convert(
    PHYLO_DIR + 'ldistance'+str(threshold)+'.xml','phyloxml',
    PHYLO_DIR + 'ldistance'+str(threshold)+'.nhx','newick')

ltree = load_tree(
    PHYLO_DIR + 'ldistance'+str(threshold)+'.nhx',
    type_='newick')

# label nodes in tree to add other attributes like subtype risk etc

In [15]:
def bandify(val,min=6.05,max=6.71,flag=False):
    maptoten=int(np.ceil(((val-min)/(max-min))*10))
    L=' '+u'\u2580'*maptoten
    
    if flag:
        L=L+u'\u21DD'
    return L

def bandify(val,min=6,max=8,flag=False):
    maptoten=int(np.ceil(((val-min)/(max-min))*10))
    L=' '+u'\u2501'*maptoten
    if flag:
        L=L+u'\u2605'
    L=L+' '
    return L

def label_nodes(
        tree, 
        recordinfo):
    """Label the nodes of the tree.
    
    We label nodes on whether:
        it is covid19
    """
    
    tree = copy.deepcopy(tree)
    
    for node in tree:
        name = node.name      
        node.subtype = recordinfo.seq_infos[name].subtype
        node.erisk =recordinfo.seq_infos[name].erisk
        node.id = recordinfo.seq_infos[name].name + bandify(recordinfo.seq_infos[name].erisk,min=threshold)
        if VERBOSE:
            print(node.name,node.subtype,node.id,node.erisk)
    return tree

# construct labelled tree

In [16]:
labelled_tree=label_nodes(
    ltree, ALLinfoHA)

# functions to collapse similar leaves

In [17]:
def prune_nodes(t):
    # collapsed nodes are labeled, so you locate them and prune them
    for n in t.search_nodes(collapsed=True):
        for ch in n.get_children():
            ch.detach()
            
            
def mean(array):
    return sum(array)/float(len(array))

def cache_distances(tree):
    ''' precalculate distances of all nodes to the root''' 
    node2rootdist = {tree:0}
    for node in tree.iter_descendants('preorder'):
        node2rootdist[node] = node.dist + node2rootdist[node.up]
    return node2rootdist

def closest_node(node, node2tips, root_distance):
    """Find the closest node."""
    
    tips = []
    distances = []
    for tip in node2tips[node]:
        distances.append(root_distance[tip]-root_distance[node])
        tips.append(tip)
        #     index = np.argmin([root_distance[tip]-root_distance[node] for tip in node2tips[node]])
    index = np.argmin(distances)
    return tips[index]

def riskiest_node(node, node2tips):
    """Find the riskiest node."""
    
    tips = []
    risks = []
    for tip in node2tips[node]:
        risks.append(tip.erisk)
        tips.append(tip)
        #     index = np.argmin([root_distance[tip]-root_distance[node] for tip in node2tips[node]])
    index = np.argmax(risks)
    return tips[index]


def all_collapsed(node, node2tips,AllrecordInfo):
    """Find all nodes in collapsed set."""
    
    tips = []
    for tip in node2tips[node]:
        tips.append(AllrecordInfo.seq_infos[tip.name].name)
    return tips

def all_collapsed(node, node2tips,AllrecordInfo):
    """Find all nodes in collapsed set."""
    
    tips = []
    for tip in node2tips.get(node, []):  # use get() to avoid KeyError if node has no entries in node2tips
        tips.append(AllrecordInfo.seq_infos[tip.name].name)
        
    # if no tips were found and the node is a leaf, append the node itself
    if not tips and node.is_leaf():
        tips.append(AllrecordInfo.seq_infos[node.name].name)

    return tips


def collapse(tree, min_dist,AllrecordInfo):
    # cache the tip content of each node to reduce the number of times the tree is traversed
    
    tree = copy.deepcopy(tree)
    
    node2tips = tree.get_cached_content()
    root_distance = cache_distances(tree)

    for node in tree.get_descendants('preorder'):
        IRAT=False
        if not node.is_leaf():
            avg_distance_to_tips = mean([root_distance[tip]-root_distance[node]
                                         for tip in node2tips[node]])
            if VERBOSE:
                print(avg_distance_to_tips)
            if avg_distance_to_tips <= min_dist:
                # do whatever, ete support node annotation, deletion, labeling, etc.
            
                #closest_name = closest_node(node, node2tips, root_distance).name
                
                # find if this name is in IRATSEQ ie any IRAT seq is in teh collapsed nosdees
                all_collapsed_nodes = all_collapsed(node, node2tips,AllrecordInfo)
                #print(all_collapsed_nodes)
                
                
                
                for i in IRATSEQ:
                    if i in all_collapsed_nodes:
                        IRAT=True
                        break
                                       
                closest_name = riskiest_node(node, node2tips).name
                node.subtype = AllrecordInfo.seq_infos[closest_name].subtype
                node.id = AllrecordInfo.seq_infos[closest_name].name + bandify(AllrecordInfo.seq_infos[closest_name].erisk,min=threshold,flag=IRAT) 
                node.name = '%s (%g)' %(closest_name,avg_distance_to_tips)
                
            
                node.add_features(collapsed=True)

                # set drawing attribute so they look collapsed when displayed with tree.show()
                node.img_style['draw_descendants'] = False
        else:
            all_collapsed_nodes = all_collapsed(node, node2tips,AllrecordInfo)
            for i in IRATSEQ:
                if i in all_collapsed_nodes:
                    IRAT=True
                    break
            node.id = AllrecordInfo.seq_infos[node.name].name + bandify(AllrecordInfo.seq_infos[node.name].erisk,min=threshold,flag=IRAT) 

    return tree

In [18]:
# collapse leaved

In [19]:
num_collapsed=20
ltree_collapsed = collapse(
    labelled_tree, 
    min_dist=num_collapsed, 
    AllrecordInfo=ALLinfoHA)

prune_nodes(ltree_collapsed)


# code for actual rendering

In [20]:
# COLBAT='DarkRed'
# COLRAT='SteelBlue'
COLH3N2='#0077FF'
COLH1N1='#551177'
COLH1N2='#337733'
COLH5N1='#991133'
COLH7N9='#aa6622'
COLH9N2='#8877FF'
COLH3N3='#FF0000'
COLDEF='#aaaaaa'

FS=50
PW=10


def nodeAttribConstruct(color, node):
    N = AttrFace(
        "id", fsize=FS, 
        text_prefix=" ",penwidth=PW,ftype='Arial',
        fgcolor=color,fstyle='bold')
    faces.add_face_to_node(N, node, 1, position="branch-right")
    return N

def layout(node):
    if node.is_leaf():
        if  node.subtype == 'H1N1':
            N = nodeAttribConstruct(COLH1N1,node)
        elif node.subtype == 'H3N2':
            N = nodeAttribConstruct(COLH3N2,node)
        elif node.subtype == 'H3N3':
            N = nodeAttribConstruct(COLH3N3,node)
        elif node.subtype == 'H7N9':
            N = nodeAttribConstruct(COLH7N9,node)
        elif node.subtype == 'H9N2':
            N = nodeAttribConstruct(COLH9N2,node)
        elif node.subtype == 'H1N2':
            N = nodeAttribConstruct(COLH1N2,node)
        elif node.subtype == 'H5N1':
            N = nodeAttribConstruct(COLH5N1,node)
        else:
            N = nodeAttribConstruct(COLDEF,node)
            



            
def render_tree(tree, outfile):# all_seq_data, display_type='nearest_host'):
    """Render the tree inside the file to a circular 
    phylogenetic tree.
    
    NOTE: outfile should be in .pdf for best visuals
    Returns:
    """
    #tree = Tree(nwfile,format=1)

    ts = TreeStyle()
    ns = NodeStyle()
    ts.show_leaf_name = False
    #ts.rotation = 90
    ts.mode = "r"
    #ts.arc_start = -360 # 0 degrees = 3 o'clock
    #ts.arc_span = 360
    ts.scale=4
    ts.show_scale=False
    ts.branch_vertical_margin = .5 # 10 pixels between adjacent branches
    # ts.show_branch_length=True
    #ts.min_leaf_separation=10
    #ts.optimal_scale_level='full'
    #ts.branch_vertical_margin=0
    
    ns.hz_line_width=2
    ns.vt_line_width=1
    #ts.layout_fn = layout
    ns["vt_line_width"] = 16
    ns["hz_line_width"] = 16
    #     ns['fsize'] = 20
    for n in tree.traverse():
        n.set_style(ns)
        
    #all_accessions = all_seq_data['accessions'].values
    for n in tree:
        ts.layout_fn = layout

        
    tree.set_style(ns)
    tree.set_style(ts)
    
    ax=tree.render(
        outfile, 
        dpi=300, 
        w=500,
        tree_style=ts)

In [21]:
r=5
medrisknames=[]
for node in ltree_collapsed:
    if ALLinfoHA.seq_infos[node.name.split()[0]].erisk > r:
        medrisknames=np.append(medrisknames,
                                ALLinfoHA.seq_infos[node.name.split()[0]].name)
medrisknames=df[df.id.isin(medrisknames)][['id','subtype',
                                               'ha_accession',
                                               'na_accession',
                                               'impact_risk',
                                               'emergence_risk']].sort_values('emergence_risk',
                                                                                         ascending=False)
medrisknames.to_csv('allriskystrains_collapsed.csv',index=None)
medrisknames[medrisknames.id.str.contains('Ohio')]

,id,subtype,ha_accession,na_accession,impact_risk,emergence_risk
3540,A/swine/Ohio/OH-21613/2021,H1N2,EPI2510646,EPI2510648,0,6.258643


# Render phylogenetic trees

In [22]:
#ax=render_tree(
#    labelled_tree,
#    './riskyphylo'+str(threshold)+'.pdf')

In [23]:
ax=render_tree(
    ltree_collapsed,
    'riskyphylo'+str(threshold)+'_collapsed_'+str(num_collapsed)+'.pdf')

/usr/share/themes/Flat-Remix-GTK-Red-Darkest/gtk-2.0/main.rc:1227: Unable to locate image file in pixmap_path: "assets/entry-background.png"
/usr/share/themes/Flat-Remix-GTK-Red-Darkest/gtk-2.0/main.rc:1228: Background image options specified without filename
/usr/share/themes/Flat-Remix-GTK-Red-Darkest/gtk-2.0/main.rc:1240: Unable to locate image file in pixmap_path: "assets/entry-background.png"
/usr/share/themes/Flat-Remix-GTK-Red-Darkest/gtk-2.0/main.rc:1241: Background image options specified without filename
Gtk-Message: 05:11:14.049: Failed to load module "pk-gtk-module"


In [24]:
ltree_collapsed

Tree node 'Inner3587' (0x7f502fdb028)

In [25]:
import dill as pickle
# Pickling (serializing) the object
with open('tree_collapsed.pkl', 'wb') as file:
    pickle.dump(ltree_collapsed, file)

# Unpickling (deserializing) the object
with open('tree_collapsed.pkl', 'rb') as file:
    ltree_collapsed = pickle.load(file)

# Find high risk strains which are on distinct branches on phylogenetic tree

In [26]:
r=6.057
r=6.5
highrisknames=[]
for node in ltree_collapsed:
    if ALLinfoHA.seq_infos[node.name.split()[0]].erisk > r:
        highrisknames=np.append(highrisknames,
                                ALLinfoHA.seq_infos[node.name.split()[0]].name)
highrisknamesdf=df[df.id.isin(highrisknames)][['id','subtype',
                                               'ha_accession',
                                               'na_accession',
                                               'impact_risk',
                                               'emergence_risk']].sort_values('emergence_risk',
                                                                                         ascending=False)
highrisknamesdf = highrisknamesdf.rename(columns={'id':'strain',
                                                  'ha_accession':'HA accession',
                                                  'na_accession':'NA accession',
                                                  'impact_risk':'predicted IRAT impact',
                                                  'emergence_risk':'predicted IRAT emergence'}).set_index('strain')
highrisknamesdf#.drop_duplicates()

,subtype,HA accession,NA accession,predicted IRAT impact,predicted IRAT emergence
strain,,,,,
A/Hong Kong/125/2017,H7N9,EPI977395,EPI977394,0,7.732746
A/California/62/2018,H1N2,EPI1311361,EPI1311360,0,7.732746
A/swine/Missouri/A02524408/2023,H3N2,EPI2433963,EPI2433964,0,7.732746
A/swine/Tver_region/RII-81-1S/2023,H1N1,EPI2965146,EPI2965145,0,7.732746
A/Indiana/08/2011,H3N2,EPI344405,EPI344404,0,7.636444
...,...,...,...,...,...
A/swine/Kansas/A02861449/2023,H3N2,EPI2609242,EPI2609238,0,6.534877
A/swine/Italy/56910/2020,H3N2,EPI2142217,EPI2142173,0,6.521545
A/chicken/China/HUNAU-GT2125/2023,H3N3,EPI2896244,EPI2896242,0,6.517744


In [42]:
print(mf[mf['emergence_risk']>7.33].index.size)
print(highrisknamesdf[highrisknamesdf['predicted IRAT emergence']>7].index.size)
print(highrisknamesdf[highrisknamesdf['predicted IRAT emergence']>7.3].index.size)


190
12
10


In [43]:
highrisknamesdf[highrisknamesdf['predicted IRAT emergence']>7.3]

,subtype,HA accession,NA accession,predicted IRAT impact,predicted IRAT emergence
strain,,,,,
A/Hong Kong/125/2017,H7N9,EPI977395,EPI977394,0,7.732746
A/California/62/2018,H1N2,EPI1311361,EPI1311360,0,7.732746
A/swine/Missouri/A02524408/2023,H3N2,EPI2433963,EPI2433964,0,7.732746
A/swine/Tver_region/RII-81-1S/2023,H1N1,EPI2965146,EPI2965145,0,7.732746
A/Indiana/08/2011,H3N2,EPI344405,EPI344404,0,7.636444
A/swine/North_Carolina/A02479173/2020,H1N1,EPI1780425,EPI1780426,0,7.599104
A/Common_Buzzard/England/125155/2023,H5N1,EPI2574053,EPI2574052,0,7.423406
A/backyard_chicken/Uruguay/UDELAR-144-M3/2023,H5N1,EPI2758928,EPI2758926,0,7.415193
A/Tufted_duck/Netherlands/1/2023,H5N1,EPI2904803,EPI2904802,0,7.412469


In [27]:
COLDICT={'H1N1':'Red3!20','H3N2':'Blue1!30','H7N9':'Green3!50','H9N2':'DarkOrange!40','H1N2':'Green2!40','H5N1':'Yellow3!40','H3N3':'Red1!40'}
def rowcolor(row):
    return '\\rowcolor{' + COLDICT[row.subtype]+'}' + row['strain']

highrisknamesdf1 = highrisknamesdf.reset_index(drop=False)
highrisknamesdf1['strain']=highrisknamesdf1.apply(rowcolor,axis=1)
highrisknamesdf1=highrisknamesdf1.set_index('strain')
highrisknamesdf1

,subtype,HA accession,NA accession,predicted IRAT impact,predicted IRAT emergence
strain,,,,,
\rowcolor{Green3!50}A/Hong Kong/125/2017,H7N9,EPI977395,EPI977394,0,7.732746
\rowcolor{Green2!40}A/California/62/2018,H1N2,EPI1311361,EPI1311360,0,7.732746
\rowcolor{Blue1!30}A/swine/Missouri/A02524408/2023,H3N2,EPI2433963,EPI2433964,0,7.732746
\rowcolor{Red3!20}A/swine/Tver_region/RII-81-1S/2023,H1N1,EPI2965146,EPI2965145,0,7.732746
\rowcolor{Blue1!30}A/Indiana/08/2011,H3N2,EPI344405,EPI344404,0,7.636444
...,...,...,...,...,...
\rowcolor{Blue1!30}A/swine/Kansas/A02861449/2023,H3N2,EPI2609242,EPI2609238,0,6.534877
\rowcolor{Blue1!30}A/swine/Italy/56910/2020,H3N2,EPI2142217,EPI2142173,0,6.521545
\rowcolor{Red1!40}A/chicken/China/HUNAU-GT2125/2023,H3N3,EPI2896244,EPI2896242,0,6.517744


In [28]:
highrisknamesdf1.reset_index().groupby('subtype').max().index.values

array(['H1N1', 'H1N2', 'H3N2', 'H3N3', 'H5N1', 'H7N9'], dtype=object)

In [29]:
from zedstat.textable import textable
textable(highrisknamesdf1,#pd.concat([highrisknamesdf1.head(30),highrisknamesdf1.tail(10)]),
         tabname='../../../../tex/overleaf3/Figures/tabdata/highrisk35_upd.tex',
         FORMAT='%1.4f',INDEX=True,
         TABFORMAT='L{1.95in}|L{.25in}|L{.60in}|L{.6in}|C{1in}|C{1in}',LNTERM='\\\\\n')
! cat ../../../../tex/overleaf3/Figures/tabdata/highrisk35_upd.tex

\begin{tabular}{L{1.95in}|L{.25in}|L{.60in}|L{.6in}|C{1in}|C{1in}}\hline
strain&subtype& HA  accession & NA  accession & predicted  IRAT  impact & predicted  IRAT  emergence \\
 \rowcolor{Green3!50}A/Hong  Kong/125/2017 &H7N9&EPI977395&EPI977394&0&7.7327\\
\rowcolor{Green2!40}A/California/62/2018&H1N2&EPI1311361&EPI1311360&0&7.7327\\
\rowcolor{Blue1!30}A/swine/Missouri/A02524408/2023&H3N2&EPI2433963&EPI2433964&0&7.7327\\
\rowcolor{Red3!20}A/swine/Tver\_region/RII-81-1S/2023&H1N1&EPI2965146&EPI2965145&0&7.7327\\
\rowcolor{Blue1!30}A/Indiana/08/2011&H3N2&EPI344405&EPI344404&0&7.6364\\
\rowcolor{Red3!20}A/swine/North\_Carolina/A02479173/2020&H1N1&EPI1780425&EPI1780426&0&7.5991\\
\rowcolor{Yellow3!40}A/Common\_Buzzard/England/125155/2023&H5N1&EPI2574053&EPI2574052&0&7.4234\\
\rowcolor{Yellow3!40}A/backyard\_chicken/Uruguay/UDELAR-144-M3/2023&H5N1&EPI2758928&EPI2758926&0&7.4152\\
\rowcolor{Yellow3!40}A/Tufted\_duck/Netherlands/1/2023&H5N1&EPI2904803&EPI2904802&0&7.4125\\
\rowcolor{Green2!40

In [30]:
#'H1N1', 'H1N2', 'H3N2', 'H3N3', 'H5N1', 'H7N9'
count=0
Subtype={'H1N1':0,'H3N2':0,'H7N9':0,'H9N2':0,'H5N1':0,'H1N2':0,'H3N3':0}
MaxriskStrain={'H1N1':None,'H3N2':None,'H7N9':None,'H9N2':None,'H5N1':None,'H1N2':None,'H3N3':None}
Subtype_strat={ 6.5:{'H1N1':0,'H3N2':0,'H7N9':0,'H9N2':0,'H5N1':0,'H1N2':0,'H3N3':0},
               7:{'H1N1':0,'H3N2':0,'H7N9':0,'H9N2':0,'H5N1':0,'H1N2':0,'H3N3':0},   
               7.5:{'H1N1':0,'H3N2':0,'H7N9':0,'H9N2':0,'H5N1':0,'H1N2':0,'H3N3':0},
               8:{'H1N1':0,'H3N2':0,'H7N9':0,'H9N2':0,'H5N1':0,'H1N2':0,'H3N3':0}}
for node in ltree_collapsed:
    Subtype[node.subtype]=Subtype[node.subtype]+1
    if MaxriskStrain[node.subtype] is None:
        MaxriskStrain[node.subtype]=(ALLinfoHA.seq_infos[node.name.split()[0]].name,
                                     ALLinfoHA.seq_infos[node.name.split()[0]].erisk)
    else:
        if MaxriskStrain[node.subtype][1]<ALLinfoHA.seq_infos[node.name.split()[0]].erisk:
            MaxriskStrain[node.subtype]=(ALLinfoHA.seq_infos[node.name.split()[0]].name,
                                         ALLinfoHA.seq_infos[node.name.split()[0]].erisk)
    for r in [6.5,7,7.5,8]:
        if r<ALLinfoHA.seq_infos[node.name.split()[0]].erisk:
            #print(node.subtype,ALLinfoHA.seq_infos[node.name.split()[0]].erisk)
            Subtype_strat[r][node.subtype]=Subtype_strat[r][node.subtype]+1
        
    count=count+1
maxriskdf = pd.DataFrame(MaxriskStrain)
Subtype_strat_df = pd.DataFrame(Subtype_strat)
display(maxriskdf)
display(Subtype_strat_df)
print(count)


,H1N1,H3N2,H7N9,H9N2,H5N1,H1N2,H3N3
0,A/swine/Tver_region/RII-81-1S/2023,A/swine/Missouri/A02524408/2023,A/Hong Kong/125/2017,A/Anhui-Lujiang/39/2018,A/Common_Buzzard/England/125155/2023,A/California/62/2018,A/chicken/China/HUNAU-GT2125/2023
1,7.732746,7.732746,7.732746,6.30493,7.423406,7.732746,6.517744


,6.5,7.0,7.5,8.0
H1N1,7,3,2,0
H3N2,14,3,2,0
H7N9,1,1,1,0
H9N2,0,0,0,0
H5N1,49,3,0,0
H1N2,8,2,1,0
H3N3,1,0,0,0


92


In [31]:
df[df.id.isin(maxriskdf.iloc[0,:].values)][['index', 'id', 'subtype', 'date', 'ha_accession', 'ha', 'na_accession',
       'na', 'emergence_risk']].sort_values('emergence_risk')

,index,id,subtype,date,ha_accession,ha,na_accession,na,emergence_risk
6,6,A/Anhui-Lujiang/39/2018,H9N2,2019-07-01,EPI1315830,METVSLITILLVATASNADKICIGYQSTNSTETVDTLTENNVPVTH...,EPI1315828,MNPNQKITAIGSVSLIIAIICLLMQIAILTTTMTLHFGQKECSNPS...,6.304930
2855,2855,A/chicken/China/HUNAU-GT2125/2023,H3N3,2023-05-11,EPI2896244,MKTIIALNYILCLAFGRNLPGKDSSTATLCLGHHSVPNGTIVKTIT...,EPI2896242,MNPNQKIITLGIVNTTLSTLALLIGIGNQIFNIVIHEKVGVHQTVV...,6.517744
126,126,A/Common_Buzzard/England/125155/2023,H5N1,2023-04-17,EPI2574053,MEXIVLLLAIVSLVKSDQICIGYHANNSTEQVDTIMEKNVTVTHAQ...,EPI2574052,MNPNQRIITTGSICMVIGIVSLMLQIGNIISIWVSHSIQTGNQYQP...,7.423406
0,0,A/Hong Kong/125/2017,H7N9,2017-05-01,EPI977395,MNTQILVFALIAIIPTNADKICLGHHAVSNGTKVNTLTERGVEVVN...,EPI977394,MNPNQKILCTSATAITIGAIAVLIGIANLGLNIGLHLKPGCNCSHS...,7.732746
2,2,A/California/62/2018,H1N2,2019-07-01,EPI1311361,MKVKLMVLLCTFTATYADTICVGYHANNSTDTVDTVLEKNVTVTHS...,EPI1311360,MNPNQKIITIGSISLTLAAMCFLMQTAILVTNVTLHFNQCECHYPP...,7.732746
24,24,A/swine/Missouri/A02524408/2023,H3N2,2023-01-13,EPI2433963,MKTIIALSNILCLVFAQKIPGNDNSTATLCLGHHAVPNGTIVKTIT...,EPI2433964,MNPNQKIITIGSVSLTISTICFFMQIAILITTVTLHFKQYEFNSPP...,7.732746
26,26,A/swine/Tver_region/RII-81-1S/2023,H1N1,2023-02-17,EPI2965146,MKAILVVMLYTFTTANADTLCIGYHANNSTDTVDTVLEKNVTVTHS...,EPI2965145,MNPNQKIITIGSICMTIGTANLILQIGNIISIWVSHSIQIGNQSQI...,7.732746


In [39]:
df[df.id=='A/swine/North_Carolina/A02479173/2020'][['index', 'id', 'subtype', 'date', 'ha_accession', 'ha', 'na_accession',
       'na', 'emergence_risk']]

,index,id,subtype,date,ha_accession,ha,na_accession,na,emergence_risk
102,102,A/swine/North_Carolina/A02479173/2020,H1N1,2020-03-25,EPI1780425,MKAILVVLLYAFTTANADTLCIGYHANNSTDTVDTVLEKNVTVTHS...,EPI1780426,MNPNQKIITIGSICMTIGTANLILQIGNIISIWVSHSIQIGNQSQI...,7.599104


In [30]:
dcf=100*(Subtype_strat_df/Subtype_strat_df.sum(axis=0))

In [48]:
df_=Subtype_strat_df
dcfs=' ('+dcf.round(2).astype(str)+'\%)'
df__=df_.astype(str)+dcfs

In [49]:
df__.columns=df__.columns.astype(str)
df__.index.name='subtype'

In [55]:
from zedstat.textable import textable
#Subtype_strat_df.columns=[str(x) for x in Subtype_strat_df.columns]
#Subtype_strat_df.index.name='subtype'
#Subtype_strat_df=Subtype_strat_df.reset_index()
textable(df__,#pd.concat([highrisknamesdf1.head(30),highrisknamesdf1.tail(10)]),
         tabname='../../../../tex/overleaf3/Figures/tabdata/riskycount_upd.tex',
         FORMAT='%s',INDEX=True,
         TABFORMAT='L{.4in}|L{.6in}|L{.6in}|L{.6in}|L{.6in}|L{.6in}',LNTERM='\\\\\n')
! cat ../../../../tex/overleaf3/Figures/tabdata/riskycount_upd.tex

\begin{tabular}{L{.4in}|L{.6in}|L{.6in}|L{.6in}|L{.6in}|L{.6in}}\hline
subtype&6.0&6.5&7.0&7.25\\
H1N1& 11  (11.46\%) & 9  (10.71\%) & 3  (23.08\%) & 2  (18.18\%) \\
H3N2& 17  (17.71\%) & 15  (17.86\%) & 4  (30.77\%) & 3  (27.27\%) \\
H7N9& 1  (1.04\%) & 1  (1.19\%) & 1  (7.69\%) & 1  (9.09\%) \\
H9N2& 1  (1.04\%) & 0  (0.0\%) & 0  (0.0\%) & 0  (0.0\%) \\
H5N1& 54  (56.25\%) & 49  (58.33\%) & 3  (23.08\%) & 3  (27.27\%) \\
H1N2& 11  (11.46\%) & 9  (10.71\%) & 2  (15.38\%) & 2  (18.18\%) \\
H3N3& 1  (1.04\%) & 1  (1.19\%) & 0  (0.0\%) & 0  (0.0\%) \\
\hline\end{tabular}


In [56]:
def write_fasta(seqs, fasta_file, wrap=80):
    """Write sequences to a fasta file.

    Parameters
    ----------
    seqs : dict[seq_id] -> seq
        Sequences indexed by sequence id.
    fasta_file : str
        Path to write the sequences to.
    wrap: int
        Number of AA/NT before the line is wrapped.
    """
    with open(fasta_file, 'w') as f:
        for gid, gseq in seqs.items():
            f.write('>{}\n'.format(gid))
            for i in range(0, len(gseq), wrap):
                f.write('{}\n'.format(gseq[i:i + wrap])) 

#for i,name in zip(df[df.id.isin(maxriskdf.iloc[0,:].values)].ha.values,
#                 df[df.id.isin(maxriskdf.iloc[0,:].values)].ha_accession.values):
#    write_fasta({name:i},name+'.fasta')
    

In [57]:
id_list=['A/swine/North_Carolina/A02479173/2020','A/swine/Tennessee/A02524414/2022']
RFILES={name+'('+subtype+')':i for i,name,subtype in zip(df[df.id.isin(id_list)].ha.values,
                 df[df.id.isin(id_list)].ha_accession.values,
                 df[df.id.isin(id_list)].subtype.values)}

In [58]:
FASTANAME='../../../../tex/overleaf3/Figures/plotdata/risky4.fasta'
FASTAFILES={name+'('+subtype+')':i for i,name,subtype in zip(df[df.id.isin(maxriskdf.iloc[0,:].values)].ha.values,
                 df[df.id.isin(maxriskdf.iloc[0,:].values)].ha_accession.values,
                 df[df.id.isin(maxriskdf.iloc[0,:].values)].subtype.values)}

In [59]:
human_dom_seq='../../../qnet_predictions/results/dominant_sequences_2021_2022.csv'
human_dom_seq='../../../../paper_data_v2/enet_predictions/dominant_sequences/results/north_h1n1_ha/north_h1n1_ha_21_22/dom_seqs.csv'
domf=pd.read_csv(human_dom_seq,index_col=0)
#domf=domf.assign(subtype=[x.split('_')[1].upper() for x in domf.index.values],protein=[x.split('_')[2].upper() for x in domf.index.values])
domf=domf.assign(subtype='H1N1',protein='HA').head(1)
domf

,name,date,sequence,acc_na,sequence_na,cluster_size,subtype,protein
acc,,,,,,,,
EPI1877316,A/Togo/0172/2021,2021-02-18,MKAILVVLLYTFTTANADTLCIGYHANNSTDTVDTVLEKNVTVTHS...,EPI1877315,MNPNQKIITIGSICMAIGTANLILQIGNIISIWVSHSIQIGNQSQI...,531,H1N1,HA


In [60]:
domf

,name,date,sequence,acc_na,sequence_na,cluster_size,subtype,protein
acc,,,,,,,,
EPI1877316,A/Togo/0172/2021,2021-02-18,MKAILVVLLYTFTTANADTLCIGYHANNSTDTVDTVLEKNVTVTHS...,EPI1877315,MNPNQKIITIGSICMAIGTANLILQIGNIISIWVSHSIQIGNQSQI...,531,H1N1,HA


In [62]:
human_dom_seq='../../../../paper_data_v0/enet_predictions/results/dominant_sequences/north_h1n1_ha/north_h1n1_ha_21_22.csv'
domf=pd.read_csv(human_dom_seq,index_col=0)
domf=domf.assign(subtype='H1N1',protein='HA')
domf=domf[domf.protein=='HA'].head(2)
DOMFILES={'DOM_HUMAN'+'('+subtype+')':i for i,name,subtype in zip(domf.sequence.values,
                 domf.id.values,
                 domf.subtype.values)}
DOMFILES.update(FASTAFILES)
DOMFILES

{'DOM_HUMAN(H1N1)': 'MKAILVVLLYTFTTANADTLCIGYHANNSTDTVDTVLEKNVTVTHSVNLLEDKHNGKLCKLRGVAPLHLGKCNIAGWILGNPECESLSTARSWSYIVETSNSDNGTCYPGDFINYEELREQLSSVSSFERFEIFPKTSSWPNHDSDKGVTAACPHAGAKSFYKNLIWLVKKGNSYPKLNQTYINDKGKEVLVLWGIHHPPTIAAQESLYQNADAYVFVGTSRYSKKFKPEIATRPKVRDQEGRMNYYWTLVEPGDKITFEATGNLVVPRYAFTMERDAGSGIIISDTPVHDCNTTCQTPEGAINTSLPFQNVHPITIGKCPKYVKSTKLRLATGLRNVPSIQSRGLFGAIAGFIEGGWTGMVDGWYGYHHQNEQGSGYAADLKSTQNAIDKITNKVNSVIEKMNTQFTAVGKEFNHLEKRIENLNKKVDDGFLDIWTYNAELLVLLENERTLDYHDSNVKNLYEKVRNQLKNNAKEIGNGCFEFYHKCDNTCMESVKNGTYDYPKYSEEAKLNREKIDGVKLESTRIYQILAIYSTVASSLVLVVSLGAISFWMCSNGSLQCRIC',
 'EPI448936(H7N9)': 'MNTQILVFALIAIIPTNADKICLGHHAVSNGTKVNTLTERGVEVVNATETVERTNIPRICSKGKRTVDLGQCGLLGTITGPPQCDQFLEFSADLIIERREGSDVCYPGKFVNEEALRQILRESGGIDKEAMGFTYSGIRTNGATSACRRSGSSFYAEMKWLLSNTDNAAFPQMTKSYKNTRKSPALIVWGIHHSVSTAEQTKLYGSGNKLVTVGSSNYQQSFVPSPGARPQVNGLSGRIDFHWLMLNPNDTVTFSFNGAFIAPDRASFLRGKSMGIQSGVQVDANCEGDCYHSGGTIISNLPFQNIDSRAVGKCPRYVKQRSLLLATGMKNVPEIPKGRGLFGAIAGFIENGWEGLIDGWYGFRHQNAQGEGTAADYKSTQSAIDQITGK

In [63]:
DOMFILES.update(RFILES)

In [64]:
DOMFILES

{'DOM_HUMAN(H1N1)': 'MKAILVVLLYTFTTANADTLCIGYHANNSTDTVDTVLEKNVTVTHSVNLLEDKHNGKLCKLRGVAPLHLGKCNIAGWILGNPECESLSTARSWSYIVETSNSDNGTCYPGDFINYEELREQLSSVSSFERFEIFPKTSSWPNHDSDKGVTAACPHAGAKSFYKNLIWLVKKGNSYPKLNQTYINDKGKEVLVLWGIHHPPTIAAQESLYQNADAYVFVGTSRYSKKFKPEIATRPKVRDQEGRMNYYWTLVEPGDKITFEATGNLVVPRYAFTMERDAGSGIIISDTPVHDCNTTCQTPEGAINTSLPFQNVHPITIGKCPKYVKSTKLRLATGLRNVPSIQSRGLFGAIAGFIEGGWTGMVDGWYGYHHQNEQGSGYAADLKSTQNAIDKITNKVNSVIEKMNTQFTAVGKEFNHLEKRIENLNKKVDDGFLDIWTYNAELLVLLENERTLDYHDSNVKNLYEKVRNQLKNNAKEIGNGCFEFYHKCDNTCMESVKNGTYDYPKYSEEAKLNREKIDGVKLESTRIYQILAIYSTVASSLVLVVSLGAISFWMCSNGSLQCRIC',
 'EPI448936(H7N9)': 'MNTQILVFALIAIIPTNADKICLGHHAVSNGTKVNTLTERGVEVVNATETVERTNIPRICSKGKRTVDLGQCGLLGTITGPPQCDQFLEFSADLIIERREGSDVCYPGKFVNEEALRQILRESGGIDKEAMGFTYSGIRTNGATSACRRSGSSFYAEMKWLLSNTDNAAFPQMTKSYKNTRKSPALIVWGIHHSVSTAEQTKLYGSGNKLVTVGSSNYQQSFVPSPGARPQVNGLSGRIDFHWLMLNPNDTVTFSFNGAFIAPDRASFLRGKSMGIQSGVQVDANCEGDCYHSGGTIISNLPFQNIDSRAVGKCPRYVKQRSLLLATGMKNVPEIPKGRGLFGAIAGFIENGWEGLIDGWYGFRHQNAQGEGTAADYKSTQSAIDQITGK

In [65]:
write_fasta(DOMFILES,FASTANAME)

In [66]:
from Bio import SeqIO
from Bio.Blast import NCBIWWW
my_query = SeqIO.read("EPI1818121.fasta", format="fasta")
result_handle = NCBIWWW.qblast("blastp", "nr", my_query.seq)
blast_result = open("my_blast.xml", "w")
blast_result.write(result_handle.read())
blast_result.close()
result_handle.close()

In [67]:
def parse_blast(resultfile): #takes in the BLAST result, outputs list that can be made into csv
    from Bio.Blast import NCBIXML
    result_handle = open(resultfile)
    blast_records = NCBIXML.parse(result_handle)
    csv_list = []
    
    header = [  'accession','Query',
                'Name', 'Length', 'Score', 'Expect',
                'QueryStart', 'QueryEnd',
                'SubjectStart', 'SubjectEnd','pct'
            ]
    
    #csv_list.append(header)
    count = 0
    for blast_record in blast_records:
        '''help(blast_record.alignments[0].hsps[0])''' # these give help info for the parts 
        '''help(blast_record.alignments[0])        '''
        count +=1
        
        query = blast_record.query
        for alignment in blast_record.alignments:

            name = alignment.title
            length = alignment.length
    
            hsp = alignment.hsps[0] # I don't know if we will ever have more than one, so might as well take the first one.
            score = hsp.score
            expect = hsp.expect
            querystart = hsp.query_start
            queryend = hsp.query_end
            subjectstart = hsp.sbjct_start
            subjectend = hsp.sbjct_end
            pct=hsp.positives/hsp.align_length
            accession=alignment.accession
            row = [accession,query,name,length,score,expect,querystart,queryend,subjectstart,subjectend,pct]
            csv_list.append(row)
            
    result_handle.close()
    return pd.DataFrame(csv_list,columns=header)
 

In [68]:
csv_list=parse_blast('my_blast.xml')
csv_list

,accession,Query,Name,Length,Score,Expect,QueryStart,QueryEnd,SubjectStart,SubjectEnd,pct
0,QIA58660,unnamed protein product,gb|QIA58660.1| hemagglutinin [Influenza A viru...,566,2969.0,0.0,1,550,1,550,1.000000
1,WHU33766,unnamed protein product,gb|WHU33766.1| hemagglutinin [Influenza A viru...,566,2967.0,0.0,1,550,1,550,1.000000
2,QKG28282,unnamed protein product,gb|QKG28282.1| hemagglutinin [Influenza A virus],566,2967.0,0.0,1,550,1,550,1.000000
3,QIZ32536,unnamed protein product,gb|QIZ32536.1| hemagglutinin [Influenza A virus],566,2967.0,0.0,1,550,1,550,1.000000
4,QJI52882,unnamed protein product,gb|QJI52882.1| hemagglutinin [Influenza A virus],566,2967.0,0.0,1,550,1,550,1.000000
5,QJT24758,unnamed protein product,gb|QJT24758.1| hemagglutinin [Influenza A virus],566,2967.0,0.0,1,550,1,550,1.000000
6,QKV49724,unnamed protein product,gb|QKV49724.1| hemagglutinin [Influenza A viru...,566,2967.0,0.0,1,550,1,550,1.000000
7,QIQ45861,unnamed protein product,gb|QIQ45861.1| hemagglutinin [Influenza A virus],566,2967.0,0.0,1,550,1,550,1.000000
8,QKM75781,unnamed protein product,gb|QKM75781.1| hemagglutinin [Influenza A virus],566,2966.0,0.0,1,550,1,550,1.000000
9,QJX57625,unnamed protein product,gb|QJX57625.1| hemagglutinin [Influenza A virus],566,2966.0,0.0,1,550,1,550,1.000000


In [69]:
from Bio import Entrez
Entrez.email = "ishanu@uchicago.edu"     # Always tell NCBI who you are
handle = Entrez.esearch(db="protein", term="QJT24340")
record = Entrez.read(handle)
id=record["IdList"]
id

['1841676337']

In [70]:
handle = Entrez.efetch(db="protein", rettype="gb", retmode="text", id="1834373412")
#record = SeqIO.read(handle, "genbank")
handle.read()

'LOCUS       QJD23117                 566 aa            linear   VRL 28-APR-2020\nDEFINITION  hemagglutinin [Influenza A virus].\nACCESSION   QJD23117\nVERSION     QJD23117.1\nDBSOURCE    accession MT372532.1\nKEYWORDS    .\nSOURCE      Influenza A virus\n  ORGANISM  Influenza A virus\n            Viruses; Riboviria; Orthornavirae; Negarnaviricota;\n            Polyploviricotina; Insthoviricetes; Articulavirales;\n            Orthomyxoviridae; Alphainfluenzavirus; Alphainfluenzavirus\n            influenzae.\nREFERENCE   1  (residues 1 to 566)\n  AUTHORS   Swine Surveillance,A.\n  TITLE     USDA NAHLN Voluntary Swine Influenza Surveillance\n  JOURNAL   Unpublished\nREFERENCE   2  (residues 1 to 566)\n  AUTHORS   Swine Surveillance,A.\n  TITLE     Direct Submission\n  JOURNAL   Submitted (23-APR-2020) DiVL, USDA NAHLN Voluntary Swine Influenza\n            Surveillance, 1920 Dayton Ave., Ames, IA 50010, USA\nCOMMENT     Method: conceptual translation.\nFEATURES             Location/Qual

H9N2: minks are probable mixing vessels
https://www.ncbi.nlm.nih.gov/nuccore/ON870694.1
 https://www.tandfonline.com/doi/full/10.1080/22221751.2021.1899058
 Pandemic influenza, typically caused by the reassortment of human and avian influenza viruses, can result in severe or fatal infections in humans. Timely identification of potential pandemic viruses must be a priority in influenza virus surveillance. However, the range of host species responsible for the generation of novel pandemic influenza viruses remains unclear. In this study, we conducted serological surveys for avian and human influenza virus infections in farmed mink and determined the susceptibility of mink to prevailing avian and human virus subtypes. The results showed that farmed mink were commonly infected with human (H3N2 and H1N1/pdm) and avian (H7N9, H5N6, and H9N2) influenza A viruses. Correlational analysis indicated that transmission of human influenza viruses occurred from humans to mink, and that feed source was a probable route of avian influenza virus transmission to farmed mink. Animal experiments showed that mink were susceptible and permissive to circulating avian and human influenza viruses, and that human influenza viruses (H3N2 and H1N1/pdm), but not avian viruses, were capable of aerosol transmission among mink. These results indicate that farmed mink could be highly permissive “mixing vessels” for the reassortment of circulating human and avian influenza viruses. 
 
 H7N9
 https://www.bmj.com/content/347/bmj.f4752?tab=responses
 HH transmission has been suspected
 Asian lineage H7N9 virus is rated by the Influenza Risk Assessment Tool as having the greatest potential to cause a pandemic, as well as potentially posing the greatest risk to severely impact public health if it were to achieve sustained human-to-human transmission.
 
 H3N2 swine
 https://www.cdc.gov/flu/swineflu/spotlights/first-human-infection-2022.htm
 Human infection detected in US this Aug
 
 H1N1
 Submitted (03-NOV-2020) USDA Swine Surveillance, USDA Swine
            Surveillance, 1920 Dayton, Ames, IA 50010, USA
COMMENT     Method: conceptual translation.



In [74]:
mf.columns

Index(['id', 'subtype', 'date', 'ha_accession', 'ha', 'na_accession', 'na',
       'ha_risk', 'na_risk', 'geom_mean_risk', 'emergence_risk',
       'emergence_low', 'emergence_high', 'is_irat', 'emergence_score',
       'impact_score', 'mean_low_acceptable_emergence',
       'mean_high_acceptable_emergence', 'mean_low_acceptable_impact',
       'mean_high_acceptable_impact', 'impact_risk'],
      dtype='object')

In [75]:
S=[np.array(x.split('/')) for x in mf[mf.emergence_risk>7].id.values]

In [76]:
S=pd.DataFrame(S,columns=['x','species','x','x','x'])

In [79]:
S.species.value_counts()/413

swine                           0.634383
Chicken                         0.046005
chicken                         0.041162
hooded_crane                    0.036320
pelican                         0.016949
                                  ...   
Shanghai                        0.002421
wildbird-Fregata-magnificens    0.002421
guanay_cormorant                0.002421
dolphin                         0.002421
turkey_vulture                  0.002421
Name: species, Length: 71, dtype: float64

In [78]:
mf[mf.emergence_risk>7]

,id,subtype,date,ha_accession,ha,na_accession,na,ha_risk,na_risk,geom_mean_risk,...,emergence_low,emergence_high,is_irat,emergence_score,impact_score,mean_low_acceptable_emergence,mean_high_acceptable_emergence,mean_low_acceptable_impact,mean_high_acceptable_impact,impact_risk
0,A/Hong Kong/125/2017,H7N9,2017-05-01,EPI977395,MNTQILVFALIAIIPTNADKICLGHHAVSNGTKVNTLTERGVEVVN...,EPI977394,MNPNQKILCTSATAITIGAIAVLIGIANLGLNIGLHLKPGCNCSHS...,0.000010,0.000010,0.000010,...,6.187663,9.179121,1,6.5,7.5,5.65,7.51,6.74,8.50,0
1,A/Shanghai/02/2013,H7N9,2016-04-01,EPI448936,MNTQILVFALIAIIPTNADKICLGHHAVSNGTKVNTLTERGVEVVN...,EPI448938,MNPNQKILCTSATAIIIGAIAVLIGMANLGLNIGLHLKPGCNCSHS...,0.000010,0.000010,0.000010,...,6.187663,9.179121,1,6.4,7.2,5.52,7.43,6.41,8.32,0
2,A/California/62/2018,H1N2,2019-07-01,EPI1311361,MKVKLMVLLCTFTATYADTICVGYHANNSTDTVDTVLEKNVTVTHS...,EPI1311360,MNPNQKIITIGSISLTLAAMCFLMQTAILVTNVTLHFNQCECHYPP...,0.000010,0.000010,0.000010,...,6.187663,9.179121,1,5.8,5.7,4.22,7.16,3.80,7.09,0
3,A/Indiana/08/2011,H3N2,2012-12-01,EPI344405,MKTIIAFSCILCLIFAQKLPGSDNSMATLCLGHHAVPNGTLVKTIT...,EPI344404,MNPNQKIITIGSVSLIIATICFLMQIAILVTTVTLHFKQHDYNSPP...,0.000055,0.000080,0.000066,...,6.104862,9.073535,1,6.0,4.5,-1.00,-1.00,-1.00,-1.00,0
23,A/swine/North_Carolina/A02751333/2022,H3N2,2022-12-13,EPI2396893,MKTIIALSNILCLVFAQKIPGNDNSTATLCLGHHAVPNGTIVKTIT...,EPI2396894,MNPNQKIITIGSVSLTISTICFFMQIAILITTVTLHFKQYEFNSPP...,0.000010,0.000010,0.000010,...,6.187663,9.179121,0,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,A/white-fronted_goose/Korea/22WC413/2023,H5N1,2023-01-16,EPI2743115,MENIVLLLATVSLVKSDQICIGYHANNSTEQVDTIMEKNVTVTHAQ...,EPI2743114,MNPNQRIITTGSICMAIGIVSLMLQIGNIISIWVSHSIQTGNQHQP...,0.005260,0.000072,0.000616,...,5.570460,8.392086,0,NaN,NaN,NaN,NaN,NaN,NaN,0
428,A/duck/Bangladesh/CEIRS-58403/2023,H5N1,2023-06-23,EPI2893639,MENIVLLLATVSLVKSDQICIGYHANNSTEQVDTIMEKNVTVTHAQ...,EPI2893637,MNPNQRIITTGSICIVIGIVSLMLQIGNIISIWVSHSIQTGNQHQP...,0.005260,0.000072,0.000616,...,5.570118,8.391651,0,NaN,NaN,NaN,NaN,NaN,NaN,0
429,A/swine/Ohio/A02685126/2023,H3N2,2023-04-10,EPI2664330,MKTIIALSYSLCLIFAQKIPGNDNGTATLCLGHHAVPNGTIVKTIT...,EPI2664328,MNPNQKIITIGSVSLTISAICFFMQIAILTTTVTLHFKQYEFNSPP...,0.000786,0.000489,0.000620,...,5.567754,8.388636,0,NaN,NaN,NaN,NaN,NaN,NaN,0
430,A/chicken/Hiroshima/22A6T/2022,H5N1,2022-12-16,EPI2747762,MENIVLLLATVSLVKSDQICIGYHANNSTEQVDTIMEKNVTVTHAQ...,EPI2747761,MNPNQRIITTGSICMAIGIVSLMLQIGNIISIWVSHSIQTGNQHQP...,0.005414,0.000072,0.000625,...,5.564128,8.384013,0,NaN,NaN,NaN,NaN,NaN,NaN,0


In [80]:
6*60/3000

0.12